In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.noise import GaussianNoise, GaussianDropout
from datetime import datetime, timedelta
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
import pyowm
import yaml

Using TensorFlow backend.


ImportError: No module named 'pyowm'

In [ ]:
with open('config.yml') as f:
    # use safe_load instead load
    config = yaml.safe_load(f)

In [ ]:
owm = pyowm.OWM(config['owm_api_key'])  # You MUST provide a valid API key
fc = owm.daily_forecast('London,uk', limit=16)
f = fc.get_forecast()
f.get_weathers()